In [15]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.base.datetools import dates_from_str


In [16]:
#Create a dateindex for the dataset

dates = dates_from_str(['2023-01-01','2023-01-02','2023-01-03','2023-01-04','2023-01-05','2023-01-06','2023-01-07','2023-01-08','2023-01-09','2023-01-10',])
num_days = len(dates)

In [21]:
#Create dummy data for the variables

np.random.seed(0)
data = np.random.rand(num_days, 5) * 100#Random values between 0 and 100
data[:, 0] += np.linspace(0,10, num_days) #Adding a linear trend to Pfizer netprice
data[:, 1] += np.linspace(0,-5, num_days) #Adding a decreasing trend to analog1 net price
data[:, 4] += np.random.randint(0, 3, size=num_days) #Random discrete values for driver



#Create a dataFRAME
columns = ['N_Pfizer', 'N_Analog1', 'N_Analog2', 'N_Analog3', 'Driver_Var']
df = pd.DataFrame(data, index=dates, columns = columns)


#Perform Granger Causality tests
max_lag = 2 #maximum lag order for granger causality tests
test_results = {}
for col in columns[:4]:
    test_results[col] = grangercausalitytests(df[[col, 'N_Pfizer']], max_lag, verbose = False)

ridge_alpha = 0.1
#Estimate VAR model
model = VAR(df)
#lag_order = max_lag #model.select_order(maxlags=2)['aic']
var_model = model.fit_regularized(alpha=ridge_alpha, maxlags=2, method='ols')


#Forecasting
forecast_steps = 3
forecast = var_model.forecast(df.values[-lag_order:], forecast_steps)



#Impulse Response Analysis
irf = var_model.irf(10)
irf.plot(orth=False)


#Display forecasted values
forecast_df = pd.DataFrame(forecast, columns=columns)
print("\n Forecasted values: \n", forecast_df)


#Print lag order selected by AIC
print("\n Selected lag order:", lag_order)


/home/cdsw/.local/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/cdsw/.local/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/cdsw/.local/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/cdsw/.local/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/cdsw/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


AttributeError: 'VAR' object has no attribute 'fit_regularized'